# Применение дообученного парафрейзера new-rut5-base-paraphraser
# на параллельном корпусе парафраз RuAdapt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
new_df = pd.read_csv('/content/drive/MyDrive/2000_random_rows.csv')
new_df

,Unnamed: 0,source,cos_sim,target
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн..."
...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч..."
1996,1225,— спросил он,0.568087,— спросил Сергей
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в..."


In [ ]:
pip install transformers sentencepiece

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/new_rut5')
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/new_rut5')

In [ ]:
def get_paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
import logging

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
%%time
new_df['output'] = new_df['source'].apply(get_paraphrase)

CPU times: user 2h 56min 18s, sys: 26.3 s, total: 2h 56min 44s
Wall time: 2h 59min 4s


In [ ]:
# Конвертация в csv-файл вместе с output
# new_df.to_csv("new_rut5_results.csv")

# Оценка результата дообученного парафрейзера new-rut5-base-paraphraser
# с помощью дистрибутивной модели rubert-base-cased-sentence

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
import gc
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

In [ ]:
new_df

,Unnamed: 0,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1225,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
%%time
DF_FILE = '/content/drive/MyDrive/new_rut5_results.csv'
# размер одного обрабатываемого кусочка датафрейма
CHUNKSIZE = 20

def process_frame(new_df):
  # функция принимает на вход датафрейм и возвращает косинусные подобия для каждого ряда
  similarities_1 = []
  for n, row in tqdm(new_df.iterrows()):
    s = row['source']
    o = row['output']
    tokens_s = tokenizer([s], return_tensors="pt", padding=True)
    tokens_o = tokenizer([o], return_tensors="pt", padding=True)
    out_s = model(**tokens_s)
    out_o = model(**tokens_o)
    vec_s = torch.mean(out_s.last_hidden_state, dim=1)
    vec_o = torch.mean(out_o.last_hidden_state, dim=1)
    cos_sim = cos(vec_s, vec_o)
    # item() превращает тензор во float
    similarities_1.append(cos_sim.item())
    # на всякий случай собираем ненужное при помощи garbage collector
    gc.collect()
  return similarities_1

# читаем файл по кусочкам заданного размера
reader = pd.read_csv(DF_FILE, chunksize=CHUNKSIZE)
# количество процессов, которые могут идти одновременно
pool = mp.Pool(4)

funclist_1 = []
for new_df_1 in reader:
  # открываем и обрабатываем каждый кусочек
  new_df.columns = ['Unnamed: 0', 'source', 'cos_sim', 'target', 'output']
  f = pool.apply_async(process_frame, [new_df_1])
  funclist_1.append(f)

result_1 = []
for f in funclist_1:
  # таймаут можно увеличить, если возникает ошибка
  sims = f.get(timeout=500)
  result_1.extend(sims)

20it [00:46,  2.33s/it]
20it [00:47,  2.38s/it]
20it [00:48,  2.43s/it]
20it [00:49,  2.48s/it]
20it [00:39,  1.99s/it]
20it [00:40,  2.04s/it]
20it [00:43,  2.16s/it]
20it [00:40,  2.03s/it]
20it [00:36,  1.84s/it]
20it [00:39,  1.96s/it]
20it [00:40,  2.03s/it]
20it [00:39,  1.99s/it]
20it [00:39,  1.99s/it]
20it [00:39,  1.98s/it]
20it [00:42,  2.12s/it]
20it [00:45,  2.27s/it]
20it [00:41,  2.05s/it]
20it [00:40,  2.03s/it]
20it [00:46,  2.34s/it]
20it [00:48,  2.44s/it]
20it [00:38,  1.93s/it]
20it [00:41,  2.07s/it]
20it [00:38,  1.93s/it]
20it [00:42,  2.12s/it]
20it [00:39,  1.99s/it]
20it [00:38,  1.94s/it]
20it [00:39,  1.98s/it]
20it [00:42,  2.12s/it]
20it [00:44,  2.21s/it]
20it [00:42,  2.11s/it]
20it [00:40,  2.04s/it]
20it [00:42,  2.10s/it]
20it [00:39,  1.96s/it]
20it [00:40,  2.00s/it]
20it [00:41,  2.07s/it]
20it [00:40,  2.03s/it]
20it [00:39,  1.97s/it]
20it [00:39,  1.98s/it]
20it [00:39,  1.97s/it]
20it [00:39,  1.99s/it]
20it [00:40,  2.02s/it]
20it [00:37,  1.

CPU times: user 11.5 s, sys: 1.55 s, total: 13.1 s
Wall time: 17min 21s


In [ ]:
result_1[:2000]

[0.9494782090187073,
 0.6755487322807312,
 0.5731340646743774,
 0.9529839754104614,
 0.8069173097610474,
 0.9069132804870605,
 0.5434597730636597,
 0.7516863942146301,
 0.7628578543663025,
 0.9145289659500122,
 0.9783755540847778,
 0.7363249063491821,
 0.744990348815918,
 0.8097272515296936,
 0.9883411526679993,
 0.9683283567428589,
 0.9542769193649292,
 0.6855109333992004,
 0.7616564631462097,
 0.7097024917602539,
 0.8811344504356384,
 0.9579476118087769,
 0.8794901967048645,
 0.9455785751342773,
 0.9443436861038208,
 0.6181284189224243,
 0.7277305722236633,
 0.987655758857727,
 0.8025479316711426,
 0.5341295599937439,
 0.8634417653083801,
 0.934636116027832,
 0.9808868765830994,
 0.9330625534057617,
 0.6682829260826111,
 0.9760515689849854,
 0.8041750192642212,
 0.963718056678772,
 0.7412348985671997,
 0.4666171371936798,
 0.7015076875686646,
 0.699943482875824,
 0.9393125176429749,
 0.6163681745529175,
 0.8770774602890015,
 0.9812794327735901,
 0.9081547260284424,
 0.873807430267334

In [ ]:
# Среднее значение
import numpy as np
np.mean(np.array(result_1))

0.8826996432989835

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(result_1))

0.09643417626074634

# METEOR
https://huggingface.co/spaces/evaluate-metric/meteor

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

meteor = load("meteor")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/new_rut5_results.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1996,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
predictions = final_df['output'].tolist()

In [ ]:
references = final_df['source'].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.3862253615246304}

# BLEU
https://huggingface.co/spaces/evaluate-metric/bleu

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

bleu = load("bleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/new_rut5_results.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1996,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['По закону моя дочь — не дочь Каренина, а дочь Каренина, а дочь Каренина, а дочь Каренина,', '«Да, всё это кончено.', 'На приглашении его двое старших тотчас же подошёл к нему.', 'Офицер бросился к её; около настасьи Филипповой уже ничего не было, и господин стоял навеселе и хохотал, что было её мочи...', 'Отродясь, я так не жил...', 'Неужели вы не помните покойного Николаевича?', 'Когда мы с ним вдвоём, он бывает в обществе очень умён и рад.', 'Она не плачет, она добрая.', 'Он так мечтал обойти его!', '— До свидания!', '— Как не узнать!', 'Этот финик тебе принять...', 'Вот что ты сделай, — продолжила она, — ты иду завтра как-можно раньше, и если получишь что-то, точас же дай мне знать!', 'До обеда оставалось ещё много времени.', '— Я устала?', 'Я была уверена, что ты на ней женишься!', '«Великие вехи забываете»', 'Донна Анна, дона Анна, дона Анна, дона Ан', 'Он пыталиво и серьёзно ещё раз посмотрел на глаза своих слушателей.', 'А я проснулся.', 'В ту самую секунду, когда Аглая начина

In [ ]:
bleu_results = bleu.compute(predictions=predictions, references=references)
print(bleu_results)

{'bleu': 0.05112770838678386, 'precisions': [0.5475796061108044, 0.16987879992052454, 0.028534426937189727, 0.00872970932427331], 'brevity_penalty': 0.7369151423735729, 'length_ratio': 0.7661176603305319, 'translation_length': 27165, 'reference_length': 35458}


# SacreBLEU
https://huggingface.co/spaces/evaluate-metric/sacrebleu

In [ ]:
pip install evaluate

In [ ]:
pip install sacrebleu

In [ ]:
from evaluate import load

sacrebleu = load("sacrebleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/new_rut5_results.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1996,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['По закону моя дочь — не дочь Каренина, а дочь Каренина, а дочь Каренина, а дочь Каренина,', '«Да, всё это кончено.', 'На приглашении его двое старших тотчас же подошёл к нему.', 'Офицер бросился к её; около настасьи Филипповой уже ничего не было, и господин стоял навеселе и хохотал, что было её мочи...', 'Отродясь, я так не жил...', 'Неужели вы не помните покойного Николаевича?', 'Когда мы с ним вдвоём, он бывает в обществе очень умён и рад.', 'Она не плачет, она добрая.', 'Он так мечтал обойти его!', '— До свидания!', '— Как не узнать!', 'Этот финик тебе принять...', 'Вот что ты сделай, — продолжила она, — ты иду завтра как-можно раньше, и если получишь что-то, точас же дай мне знать!', 'До обеда оставалось ещё много времени.', '— Я устала?', 'Я была уверена, что ты на ней женишься!', '«Великие вехи забываете»', 'Донна Анна, дона Анна, дона Анна, дона Ан', 'Он пыталиво и серьёзно ещё раз посмотрел на глаза своих слушателей.', 'А я проснулся.', 'В ту самую секунду, когда Аглая начина

In [ ]:
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(sacrebleu_results)
print(round(sacrebleu_results["score"], 1))

{'score': 5.1127708386783866, 'counts': [14875, 4275, 661, 185], 'totals': [27165, 25165, 23165, 21192], 'precisions': [54.757960611080435, 16.987879992052452, 2.8534426937189727, 0.872970932427331], 'bp': 0.7369151423735729, 'sys_len': 27165, 'ref_len': 35458}
5.1


# ROUGE
https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [ ]:
from evaluate import load

rouge = load("rouge")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/new_rut5_results.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1996,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['По закону моя дочь — не дочь Каренина, а дочь Каренина, а дочь Каренина, а дочь Каренина,', '«Да, всё это кончено.', 'На приглашении его двое старших тотчас же подошёл к нему.', 'Офицер бросился к её; около настасьи Филипповой уже ничего не было, и господин стоял навеселе и хохотал, что было её мочи...', 'Отродясь, я так не жил...', 'Неужели вы не помните покойного Николаевича?', 'Когда мы с ним вдвоём, он бывает в обществе очень умён и рад.', 'Она не плачет, она добрая.', 'Он так мечтал обойти его!', '— До свидания!', '— Как не узнать!', 'Этот финик тебе принять...', 'Вот что ты сделай, — продолжила она, — ты иду завтра как-можно раньше, и если получишь что-то, точас же дай мне знать!', 'До обеда оставалось ещё много времени.', '— Я устала?', 'Я была уверена, что ты на ней женишься!', '«Великие вехи забываете»', 'Донна Анна, дона Анна, дона Анна, дона Ан', 'Он пыталиво и серьёзно ещё раз посмотрел на глаза своих слушателей.', 'А я проснулся.', 'В ту самую секунду, когда Аглая начина

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split())
rouge_results

{'rouge1': 0.3434432622868349,
 'rouge2': 0.08056362942520409,
 'rougeL': 0.3295405679846145,
 'rougeLsum': 0.32903120008337816}

# BERTScore
https://huggingface.co/spaces/evaluate-metric/bertscore

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/new_rut5_results.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1996,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['По закону моя дочь — не дочь Каренина, а дочь Каренина, а дочь Каренина, а дочь Каренина,', '«Да, всё это кончено.', 'На приглашении его двое старших тотчас же подошёл к нему.', 'Офицер бросился к её; около настасьи Филипповой уже ничего не было, и господин стоял навеселе и хохотал, что было её мочи...', 'Отродясь, я так не жил...', 'Неужели вы не помните покойного Николаевича?', 'Когда мы с ним вдвоём, он бывает в обществе очень умён и рад.', 'Она не плачет, она добрая.', 'Он так мечтал обойти его!', '— До свидания!', '— Как не узнать!', 'Этот финик тебе принять...', 'Вот что ты сделай, — продолжила она, — ты иду завтра как-можно раньше, и если получишь что-то, точас же дай мне знать!', 'До обеда оставалось ещё много времени.', '— Я устала?', 'Я была уверена, что ты на ней женишься!', '«Великие вехи забываете»', 'Донна Анна, дона Анна, дона Анна, дона Ан', 'Он пыталиво и серьёзно ещё раз посмотрел на глаза своих слушателей.', 'А я проснулся.', 'В ту самую секунду, когда Аглая начина

In [ ]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
bertscore_results

{'precision': [0.9390625357627869,
  0.9691019654273987,
  0.9425345063209534,
  0.9277826547622681,
  0.9240446090698242,
  0.9629466533660889,
  0.955877959728241,
  0.9737839698791504,
  0.9672304391860962,
  0.9825970530509949,
  0.96934974193573,
  0.9168655872344971,
  0.9318491220474243,
  0.954999566078186,
  0.9755759835243225,
  0.9700189828872681,
  0.9687870740890503,
  0.9099775552749634,
  0.9067890644073486,
  0.9282737970352173,
  0.9227093458175659,
  0.9722590446472168,
  0.948759138584137,
  0.9544175863265991,
  0.9697477221488953,
  0.9395855665206909,
  0.9596987366676331,
  0.9694479703903198,
  0.9368637800216675,
  0.9439159035682678,
  0.9312266707420349,
  0.9549961090087891,
  0.9520202875137329,
  0.9885017275810242,
  0.9293279647827148,
  0.9682215452194214,
  0.9281843304634094,
  0.9568088054656982,
  0.9715716242790222,
  0.963021993637085,
  0.8715166449546814,
  0.9484924674034119,
  0.9587997198104858,
  0.9174274802207947,
  0.9462141990661621,
  1

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['precision']))

0.9480499427616597

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['recall']))

0.9242296218276024

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['f1']))

0.935594124853611

# ParaScore
https://github.com/shadowkiller33/ParaScore

In [ ]:
pip install parascore==1.0.5

In [ ]:
from parascore import ParaScorer

In [ ]:
scorer = ParaScorer(lang="ru", model_type='bert-base-uncased')

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/new_rut5_results.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"По закону моя дочь — не дочь Каренина, а дочь ..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","«Да, всё это кончено."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших тотчас же подо...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Офицер бросился к её; около настасьи Филиппово...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродясь, я так не жил..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, и в ночь он уехал к себя."
1996,1996,— спросил он,0.568087,— спросил Сергей,— спросила она. — Что я хочу сказать?
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Мне нужно было увидеть то счастье, которое нап..."


In [ ]:
cands = final_df['output'].tolist()
sources = final_df['source'].tolist()
refs = final_df['target'].tolist()

In [ ]:
parascore_results = scorer.base_score(cands, sources, refs, batch_size=16)
parascore_results

[0.8867671656608581,
 0.82228435754776,
 0.8656485843658447,
 0.829024212360382,
 0.7645887899398803,
 0.8820083904266357,
 0.8523301649093628,
 0.8447501230239868,
 0.8501880931854248,
 0.9419023626191275,
 0.9423787406512669,
 0.8765256571769714,
 0.8208481836318969,
 1.0034405159950257,
 0.9733970928192138,
 0.9110486498245827,
 0.9074353541646685,
 0.8235534596443176,
 0.7748632597923278,
 0.844582083273907,
 0.824191288948059,
 0.9058644698216365,
 0.8940305192129953,
 0.8935458301151626,
 0.9035729680742536,
 0.8170359301567077,
 0.8654478359222412,
 0.9475516387394496,
 0.9202180075645446,
 0.8727619076261715,
 0.8199557828903198,
 0.9359285998344421,
 0.9210810232162475,
 0.9420566865376063,
 0.8447958993911743,
 0.9260349440574646,
 0.8758616614341735,
 0.8819141500956053,
 0.9097002911567688,
 0.8493243026733398,
 0.7693391609191894,
 0.7382235693931579,
 0.9031300776345389,
 0.8544660973548889,
 0.8284333515167236,
 0.9714285714285714,
 0.7874571847915649,
 0.784548299312591

In [ ]:
import numpy as np
np.mean(np.array(parascore_results))

0.87433526113521

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(parascore_results))

0.05839011490184463